In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [2]:
# STEP 1: Load dataset
df = pd.read_csv(r'D:\JOB\1 Pricing analyst\AB testing\Projects\2 Multivariate Test (MVT)\mvt_pricing_data.csv')

In [14]:
df

,user_id,price_point,discount_percent,marketing_channel,device,location,gender,age,session_duration,pages_viewed,conversion,revenue,combo
0,1,24.99,5,social,desktop,UK,male,40,89.75,3,0,0.00,P24.99_D5_Csocial
1,2,29.99,15,email,desktop,UK,male,32,177.33,6,0,0.00,P29.99_D15_Cemail
2,3,19.99,20,email,desktop,Canada,female,26,68.27,2,0,0.00,P19.99_D20_Cemail
3,4,29.99,15,email,desktop,UK,other,57,371.60,0,0,0.00,P29.99_D15_Cemail
4,5,29.99,15,search,mobile,Canada,male,55,718.36,6,0,0.00,P29.99_D15_Csearch
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,14.99,15,email,desktop,Germany,female,42,1074.18,5,1,14.99,P14.99_D15_Cemail
9996,9997,19.99,15,social,desktop,Germany,other,27,109.69,8,0,0.00,P19.99_D15_Csocial
9997,9998,14.99,5,email,mobile,UK,other,22,274.09,3,1,14.99,P14.99_D5_Cemail
9998,9999,24.99,5,social,mobile,UK,other,49,512.49,3,0,0.00,P24.99_D5_Csocial


In [3]:
# STEP 2: Create revenue column
df['revenue'] = df['conversion'] * df['price_point']

In [4]:
# STEP 3: Group by combinations (MVT)
conversion_combo = df.groupby(['price_point', 'discount_percent', 'marketing_channel'])['conversion'].mean().reset_index()
conversion_combo.columns = ['price_point', 'discount_percent', 'marketing_channel', 'conversion_rate']

In [5]:
revenue_combo = df.groupby(['price_point', 'discount_percent', 'marketing_channel'])['revenue'].mean().reset_index()
revenue_combo.columns = ['price_point', 'discount_percent', 'marketing_channel', 'revenue_per_user']

In [6]:
mvt_summary = pd.merge(conversion_combo, revenue_combo, on=['price_point', 'discount_percent', 'marketing_channel'])
mvt_summary['combo'] = (
    'Price: $' + mvt_summary['price_point'].astype(str) + ' | ' +
    'Discount: ' + mvt_summary['discount_percent'].astype(str) + '% | ' +
    'Channel: ' + mvt_summary['marketing_channel']
)

In [7]:
mvt_summary.nlargest(10, 'revenue_per_user')

,price_point,discount_percent,marketing_channel,conversion_rate,revenue_per_user,combo
106,29.99,5,direct,0.208333,6.247917,Price: $29.99 | Discount: 5% | Channel: direct
116,29.99,15,direct,0.205128,6.151795,Price: $29.99 | Discount: 15% | Channel: direct
90,24.99,15,affiliate,0.234568,5.861852,Price: $24.99 | Discount: 15% | Channel: affil...
114,29.99,10,social,0.184615,5.536615,Price: $29.99 | Discount: 10% | Channel: social
82,24.99,5,email,0.221053,5.524105,Price: $24.99 | Discount: 5% | Channel: email
112,29.99,10,email,0.178082,5.340685,Price: $29.99 | Discount: 10% | Channel: email
107,29.99,5,email,0.173333,5.198267,Price: $29.99 | Discount: 5% | Channel: email
87,24.99,10,email,0.202532,5.061266,Price: $24.99 | Discount: 10% | Channel: email
77,24.99,0,email,0.190476,4.760000,Price: $24.99 | Discount: 0% | Channel: email
96,24.99,20,direct,0.190476,4.760000,Price: $24.99 | Discount: 20% | Channel: direct


In [8]:
# STEP 5: Prepare for logistic regression
df['combo'] = (
    'P' + df['price_point'].astype(str) +
    '_D' + df['discount_percent'].astype(str) +
    '_C' + df['marketing_channel']
)

In [9]:
# STEP 6: One-hot encode categorical variables
df_encoded = pd.get_dummies(df, columns=[
    'price_point', 'discount_percent', 'marketing_channel',
    'device', 'location', 'gender'
], drop_first=True)

In [10]:
# STEP 7: Define features and target
X = df_encoded.drop(columns=['user_id', 'conversion', 'combo', 'revenue'])
y = df_encoded['conversion']

In [11]:
# STEP 8: Add constant and ensure numeric types
X = sm.add_constant(X)
X = X.apply(lambda col: col.astype(int) if col.dtype == 'bool' else col)
X = X.apply(pd.to_numeric)

In [12]:
# STEP 9: Fit logistic regression
model = sm.Logit(y, X).fit(disp=False)

In [13]:
# STEP 10: Show summary
print(model.summary())

                           Logit Regression Results                           
Dep. Variable:             conversion   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9976
Method:                           MLE   Df Model:                           23
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                0.002430
Time:                        12:36:46   Log-Likelihood:                -4213.4
converged:                       True   LL-Null:                       -4223.6
Covariance Type:            nonrobust   LLR p-value:                    0.6098
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       -1.7723      0.166    -10.652      0.000      -2.098      -1.446
age                          0.0026      0.002      1.228      0.219      -0.002       0.